In [3]:
import pandas as pd
import numpy as np
import datetime
import cx_Oracle
import config
import time


In [4]:
'''Conexión a la base de datos'''
cx_Oracle.init_oracle_client(lib_dir=config.lib_dir)
connection = None
try:
    connection = cx_Oracle.connect(
        config.username,
        config.password,
        config.dsn,
        encoding=config.encoding)
    # show the version of the Oracle Database
    print(connection.version)
except cx_Oracle.Error as error:
    print("error:", error)


ProgrammingError: Oracle Client library has already been initialized

### Consultas Personas Retiradas

In [5]:
# numero de asignaciones facturadas x persona
time_start = datetime.datetime.now()

cur = connection.cursor()
cur.execute("\
SELECT P.CONSECUTIVO AS PERSONA, CASE WHEN  H.ASIGNACIONES_FACTURADAS IS NULL THEN 0 ELSE H.ASIGNACIONES_FACTURADAS END AS ASIGNACIONES_FACTURADAS \
FROM PERSONAS P \
LEFT JOIN ( \
    SELECT P.CONSECUTIVO AS PERSONA, COUNT(A.TARIFA_HORA) AS ASIGNACIONES_FACTURADAS \
    FROM ASIGNACIONES A \
    INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO = AH.CON_ASIG \
    INNER JOIN PERSONAS P                 ON P.CONSECUTIVO = A.CON_PERSONA \
    WHERE AH.FECHA_FIN > ADD_MONTHS(P.FECHA_RETIRO, -14) \
    AND   AH.FECHA_FIN < ADD_MONTHS(P.FECHA_RETIRO, -2) \
    AND   AH.FECHA_FIN > AH.FECHA_INICIO \
    AND   A.TARIFA_HORA > 0 \
    GROUP BY P.CONSECUTIVO) H ON P.CONSECUTIVO = H.PERSONA \
WHERE P.FECHA_RETIRO IS NOT NULL \
ORDER BY P.CONSECUTIVO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_facturas_persona = pd.DataFrame(
    res, columns=['PERSONA', 'ASIGNACIONES_FACTURADAS'])
df_facturas_persona.head(2)


Duración de Consulta (seg): 0:00:20.594850


,PERSONA,ASIGNACIONES_FACTURADAS
0,2,0
1,7,1


In [ ]:
# numero de asignaciones no facturadas x persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT P.CONSECUTIVO AS PERSONA, CASE WHEN  H.ASIGNACIONES_NO_FACTURADAS IS NULL THEN 0 ELSE H.ASIGNACIONES_NO_FACTURADAS END AS ASIGNACIONES_NO_FACTURADAS \
FROM PERSONAS P \
LEFT JOIN ( \
    SELECT P.CONSECUTIVO AS PERSONA, COUNT(A.TARIFA_HORA) AS ASIGNACIONES_NO_FACTURADAS \
    FROM ASIGNACIONES A \
    INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO = AH.CON_ASIG \
    INNER JOIN PERSONAS P                 ON P.CONSECUTIVO = A.CON_PERSONA \
    WHERE AH.FECHA_FIN > ADD_MONTHS(P.FECHA_RETIRO, -14) \
    AND   AH.FECHA_FIN < ADD_MONTHS(P.FECHA_RETIRO, -2) \
    AND   AH.FECHA_FIN > AH.FECHA_INICIO \
    AND   A.TARIFA_HORA = 0 \
    GROUP BY P.CONSECUTIVO) H ON P.CONSECUTIVO = H.PERSONA \
 WHERE P.FECHA_RETIRO IS NOT NULL \
ORDER BY P.CONSECUTIVO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_nofacturas_persona = pd.DataFrame(
    res, columns=['PERSONA', 'ASIGNACIONES_NO_FACTURADAS'])
df_nofacturas_persona.head(2)


Duración de Consulta (seg): 0:00:03.264900


,PERSONA,ASIGNACIONES_NO_FACTURADAS
0,2,0
1,7,6


In [ ]:
# Asignaciones de personas por tipos de servicio
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.CANTIDAD_TIPOS_SERVICIO IS NULL THEN 0 ELSE B.CANTIDAD_TIPOS_SERVICIO END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NOT NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.TIPO AS TIPO_SERVICIO, COUNT(S.TIPO) AS CANTIDAD_TIPOS_SERVICIO \
	FROM ASIGNACIONES A \
	INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO  = AH.CON_ASIG \
	INNER JOIN PERSONAS P                 ON A.CON_PERSONA  = P.CONSECUTIVO \
	INNER JOIN SERVICIOS S                ON A.COD_SERVICIO = S.CODIGO_SERVICIO \
	WHERE AH.FECHA_FIN > ADD_MONTHS(P.FECHA_RETIRO,-14) \
	AND   AH.FECHA_FIN < ADD_MONTHS(P.FECHA_RETIRO ,-2) \
	AND   AH.FECHA_FIN >= AH.FECHA_INICIO \
	AND   P.FECHA_RETIRO IS NOT NULL \
	GROUP BY P.CONSECUTIVO, S.TIPO ) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA,A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_tiposervicio_persona = pd.DataFrame(
    res, columns=['PERSONA', 'TIPO_SERVICIO', 'CANTIDAD_TIPOS_SERVICIO'])
pv_tiposervicio_persona = pd.pivot_table(df_tiposervicio_persona, values='CANTIDAD_TIPOS_SERVICIO', index=[
                                         'PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_tiposervicio_persona.head(2)


Duración de Consulta (seg): 0:00:25.901487


TIPO_SERVICIO,AC,ADM,AI,AP,CO,CU,GC,HD,IN,LI,NE,OP,PD,PM,PR,RE,SE,SM,SO,TD
PERSONA,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,11,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0


In [ ]:
# Horas de personas por tipos de servicio
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.HORAS_TIPO_SERVICIO IS NULL THEN 0 ELSE B.HORAS_TIPO_SERVICIO END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NOT NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT \
	P.CONSECUTIVO          AS PERSONA, \
	S.TIPO                 AS TIPO_SERVICIO, \
	SUM(EA.DURACION_HORAS) AS HORAS_TIPO_SERVICIO \
	FROM EJECUCION_ACTIVIDADES EA \
	INNER JOIN ENTREGABLES_SERVICIO ES ON EA.CON_ENTREGABLE_SERV = ES.CONSECUTIVO \
	INNER JOIN SERVICIOS S             ON ES.COD_SERVICIO        = S.CODIGO_SERVICIO \
	INNER JOIN PERSONAS  P             ON EA.CONSECUTIVO_PERSONA = P.CONSECUTIVO \
	WHERE EA.HORA_FINALIZACION > ADD_MONTHS(P.FECHA_RETIRO, -14) \
	AND   EA.HORA_FINALIZACION < ADD_MONTHS(P.FECHA_RETIRO , -2) \
	GROUP BY P.CONSECUTIVO, S.TIPO ) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA,A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_horas_tiposervicio_persona = pd.DataFrame(
    res, columns=['PERSONA', 'TIPO_SERVICIO', 'HORAS_TIPO_SERVICIO'])
pv_horas_tiposervicio_persona = pd.pivot_table(df_horas_tiposervicio_persona, values='HORAS_TIPO_SERVICIO', index=[
                                               'PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_horas_tiposervicio_persona.head(2)


Duración de Consulta (seg): 0:00:29.696168


TIPO_SERVICIO,AC,ADM,AI,AP,CO,CU,GC,HD,IN,LI,NE,OP,PD,PM,PR,RE,SE,SM,SO,TD
PERSONA,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0
7,0.0,28.0,0.0,0.0,0.0,0,0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0,0.0,0.0,2256.0,0


In [ ]:
# Tecnologias usada en asignaciones por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.HERRAMIENTA, CASE WHEN B.VECES_HERRAMIENTA_ASIGNADA IS NULL THEN 0 ELSE B.VECES_HERRAMIENTA_ASIGNADA END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA,H.NOMBRE AS HERRAMIENTA \
	FROM HERRAMIENTAS H \
	CROSS JOIN PERSONAS P \
    WHERE    P.FECHA_RETIRO IS NOT NULL\
	GROUP BY P.CONSECUTIVO,H.NOMBRE) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO  AS PERSONA,H.NOMBRE  AS HERRAMIENTA,COUNT(AH.CONSECUTIVO) AS VECES_HERRAMIENTA_ASIGNADA \
	FROM HERRAMIENTAS_UTILIZADAS HU \
	INNER JOIN HERRAMIENTAS H             ON HU.CON_HERRAMIENTAS = H.CONSECUTIVO \
	INNER JOIN SERVICIOS S                ON HU.COD_SERVICIO     = S.CODIGO_SERVICIO \
	INNER JOIN ASIGNACIONES A             ON S.CODIGO_SERVICIO   = A.COD_SERVICIO \
	INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO       = AH.CON_ASIG \
	INNER JOIN PERSONAS P                 ON P.CONSECUTIVO       = A.CON_PERSONA \
	WHERE AH.FECHA_FIN > ADD_MONTHS(P.FECHA_RETIRO, -14) \
	AND   AH.FECHA_FIN < ADD_MONTHS(P.FECHA_RETIRO, -2) \
	AND   AH.FECHA_FIN > AH.FECHA_INICIO \
	AND   P.FECHA_RETIRO IS NOT NULL \
	GROUP BY P.CONSECUTIVO, H.NOMBRE) B ON A.PERSONA = B.PERSONA AND  A.HERRAMIENTA = B.HERRAMIENTA \
ORDER BY A.PERSONA, A.HERRAMIENTA")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_tecno_asig_persona = pd.DataFrame(
    res, columns=['PERSONA', 'HERRAMIENTA', 'VECES_HERRAMIENTA_ASIGNADA'])
pv_tecno_asig_persona = pd.pivot_table(df_tecno_asig_persona, values='VECES_HERRAMIENTA_ASIGNADA', index=[
                                       'PERSONA'], columns='HERRAMIENTA', fill_value=0)
pv_tecno_asig_persona.head(2)


Duración de Consulta (seg): 0:03:06.536473


HERRAMIENTA,.NET,.NET Core,ANGULAR,AWS,Activemq,Alibaba,Android / Java,Android / Kotlin,Angular,Apache Airflow,...,Travis CI,TypeScript,Vue,WEBLOGIC,Websphere,Websphere Message Broker,Wordpress,Xamarin,iOS / Objetive C,iOS / Swift
PERSONA,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Horas por etapa por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.ETAPA, CASE WHEN B.HORAS_ETAPA IS NULL THEN 0 ELSE B.HORAS_ETAPA END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, E.CODIGO_ETAPA  AS ETAPA \
	FROM ETAPAS  E \
	CROSS JOIN PERSONAS P \
    WHERE  P.FECHA_RETIRO IS NOT NULL\
	GROUP BY P.CONSECUTIVO,E.CODIGO_ETAPA) A \
LEFT JOIN ( \
	SELECT \
	P.CONSECUTIVO          AS PERSONA, \
	EA.CODIGO_ETAPA        AS ETAPA, \
	SUM(EA.DURACION_HORAS) AS HORAS_ETAPA \
	FROM EJECUCION_ACTIVIDADES EA \
	INNER JOIN PERSONAS P ON EA.CONSECUTIVO_PERSONA = P.CONSECUTIVO \
	WHERE EA.HORA_FINALIZACION > ADD_MONTHS(P.FECHA_RETIRO, -14) \
	AND   EA.HORA_FINALIZACION < ADD_MONTHS(P.FECHA_RETIRO, -2) \
	GROUP BY P.CONSECUTIVO, EA.CODIGO_ETAPA ) B ON A.PERSONA = B.PERSONA AND A.ETAPA = B.ETAPA \
ORDER BY A.PERSONA, A.ETAPA")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_horas_etapa_persona = pd.DataFrame(
    res, columns=['PERSONA', 'ETAPA', 'HORAS_ETAPA'])
df_horas_etapa_persona['ETAPA'] = df_horas_etapa_persona['ETAPA'] .fillna(
    'SIN_ETAPA')
pv_horas_etapa_persona = pd.pivot_table(df_horas_etapa_persona, values='HORAS_ETAPA', index=[
                                        'PERSONA'], columns='ETAPA', fill_value=0)
pv_horas_etapa_persona.head(2)


Duración de Consulta (seg): 0:07:03.288865


ETAPA,ACTCOM,ADMB,ADMBD,ADMCAP,ADMON,ADMSEG,ADMSER,ADMSO,ALEMAN,ANAID,...,TRIBUT,TRINS,TRINSM,TRINT,TRMIG,TRREV,VAC,VAR,VENPRO,VISCLI
PERSONA,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.00,0,0.0,0,0.0,0.0,0,0.0,...,0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0
7,0.0,4.0,1881.77,0,0.0,0,0.0,0.0,0,0.0,...,0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0


In [ ]:
# Horas en area por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT B.PERSONA, B.AREA, CASE WHEN C.HORAS_AREA IS NULL THEN 0 ELSE C.HORAS_AREA END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, A.CONSECUTIVO  AS AREA \
	FROM AREAS  A \
	CROSS JOIN PERSONAS P \
    WHERE P.FECHA_RETIRO IS NOT NULL \
	GROUP BY P.CONSECUTIVO,A.CONSECUTIVO  ) B \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.CON_AREA  AS AREA, SUM(EA.DURACION_HORAS) AS HORAS_AREA \
	FROM EJECUCION_ACTIVIDADES EA \
	INNER JOIN ENTREGABLES_SERVICIO ES ON EA.CON_ENTREGABLE_SERV = ES.CONSECUTIVO \
	INNER JOIN PERSONAS P              ON EA.CONSECUTIVO_PERSONA = P.CONSECUTIVO \
	INNER JOIN SERVICIOS S             ON ES.COD_SERVICIO  = S.CODIGO_SERVICIO \
	WHERE EA.HORA_FINALIZACION > ADD_MONTHS(P.FECHA_RETIRO, -14) \
	AND   EA.HORA_FINALIZACION < ADD_MONTHS(P.FECHA_RETIRO, -2) \
	GROUP BY P.CONSECUTIVO, S.CON_AREA ) C ON B.PERSONA = C.PERSONA AND C.PERSONA =  B.AREA \
ORDER BY B.PERSONA, B.AREA")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_horas_area_persona = pd.DataFrame(
    res, columns=['PERSONA', 'AREA', 'HORAS_AREA'])
pv_horas_area_persona = pd.pivot_table(df_horas_area_persona, values='HORAS_AREA', index=[
                                       'PERSONA'], columns='AREA', fill_value=0)
pv_horas_area_persona.head(2)


Duración de Consulta (seg): 0:00:54.930705


AREA,1,2,3,4,5,6,7,8,9,24,...,53,54,55,56,58,59,60,61,62,63
PERSONA,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0.00,0,0.0,0,...,0,0,0,0.0,0,0.0,0,0,0,0
7,0,0,0,0,0,0,571.25,0,0.0,0,...,0,0,0,0.0,0,0.0,0,0,0,0


In [ ]:
# Cantidad de pendientes en tipo de servicio por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.CANTIDAD_PENDIENTES IS NULL THEN 0 ELSE B.CANTIDAD_PENDIENTES END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NOT NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.TIPO AS TIPO_SERVICIO, COUNT(PE.CODIGO_PENDIENTE) AS CANTIDAD_PENDIENTES \
	FROM ASIGNACIONES_HISTORICOS AH \
	INNER JOIN ASIGNACIONES A  ON A.CONSECUTIVO = AH.CON_ASIG \
	INNER JOIN PERSONAS     P  ON A.CON_PERSONA = P.CONSECUTIVO \
	INNER JOIN PENDIENTES   PE ON A.COD_SERVICIO = PE.CODIGO_SERVICIO \
	INNER JOIN SERVICIOS    S  ON PE.CODIGO_SERVICIO = S.CODIGO_SERVICIO \
	WHERE AH.FECHA_FIN > ADD_MONTHS(P.FECHA_RETIRO, -14) \
	AND   AH.FECHA_FIN < ADD_MONTHS(P.FECHA_RETIRO, -2) \
	AND   AH.FECHA_FIN >= AH.FECHA_INICIO \
	AND   PE.FECHA_DETECCION > ADD_MONTHS(P.FECHA_RETIRO, -14) \
	AND   PE.FECHA_DETECCION < ADD_MONTHS(P.FECHA_RETIRO, -2) \
	GROUP BY P.CONSECUTIVO, S.TIPO) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA,A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_pend_tiposerv_persona = pd.DataFrame(
    res, columns=['PERSONA', 'TIPO_SERVICIO', 'CANTIDAD_PENDIENTES'])
pv_pend_tiposerv_persona = pd.pivot_table(df_pend_tiposerv_persona, values='CANTIDAD_PENDIENTES', index=[
                                          'PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_pend_tiposerv_persona.head(2)


Duración de Consulta (seg): 0:00:35.822878


TIPO_SERVICIO,AC,ADM,AI,AP,CO,CU,GC,HD,IN,LI,NE,OP,PD,PM,PR,RE,SE,SM,SO,TD
PERSONA,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,261,0,0,0,0,0,0,0,0,0,0,0,29,0,0,0,0,0,0


In [ ]:
# Cantidad de riesgos en tipo de servicio por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.CANTIDAD_RIESGOS IS NULL THEN 0 ELSE B.CANTIDAD_RIESGOS END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NOT NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.TIPO AS TIPO_SERVICIO, COUNT(RS.CONSECUTIVO) AS CANTIDAD_RIESGOS \
	FROM ASIGNACIONES_HISTORICOS AH \
	INNER JOIN ASIGNACIONES A      ON A.CONSECUTIVO = AH.CON_ASIG \
	INNER JOIN PERSONAS     P      ON A.CON_PERSONA = P.CONSECUTIVO \
	INNER JOIN RIESGOS_SERVICIO RS ON A.COD_SERVICIO = RS.CODIGO_SERVICIO \
	INNER JOIN SERVICIOS    S      ON RS.CODIGO_SERVICIO = S.CODIGO_SERVICIO \
	WHERE AH.FECHA_FIN BETWEEN ADD_MONTHS(P.FECHA_RETIRO, -14) AND ADD_MONTHS(P.FECHA_RETIRO, -2) \
	AND   AH.FECHA_FIN > AH.FECHA_INICIO \
	AND   RS.FECHA_REGISTRO BETWEEN ADD_MONTHS(P.FECHA_RETIRO, -14)  AND ADD_MONTHS(P.FECHA_RETIRO, -2) \
	GROUP BY P.CONSECUTIVO, S.TIPO ) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA, A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_ries_tiposerv_persona = pd.DataFrame(
    res, columns=['PERSONA', 'TIPO_SERVICIO', 'CANTIDAD_RIESGOS'])
pv_ries_tiposerv_persona = pd.pivot_table(df_ries_tiposerv_persona, values='CANTIDAD_RIESGOS', index=[
                                          'PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_ries_tiposerv_persona.head(2)


Duración de Consulta (seg): 0:00:27.834556


TIPO_SERVICIO,AC,ADM,AI,AP,CO,CU,GC,HD,IN,LI,NE,OP,PD,PM,PR,RE,SE,SM,SO,TD
PERSONA,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Creación Dataset Retirados

In [ ]:
time_start = datetime.datetime.now()

print('asignaciones facturadas x persona:', df_facturas_persona.shape)
print('asignaciones no facturadas x persona:', df_nofacturas_persona.shape)
print('numero tipos servicio x persona:', pv_tiposervicio_persona.shape)
print('horas tipo servicio x persona:', pv_horas_tiposervicio_persona.shape)
print('tecnologias usadas x persona:', pv_tecno_asig_persona.shape)
print('horas en etapa x persona:', pv_horas_etapa_persona.shape)
print('horas en area x persona:', pv_horas_area_persona.shape)
print('pendientes en tipos de servicio x persona:',
      pv_pend_tiposerv_persona.shape)
print('riesgos en tipos de servicio x persona:', pv_ries_tiposerv_persona.shape)

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")


asignaciones facturadas x persona: (1378, 2)
asignaciones no facturadas x persona: (1378, 2)
numero tipos servicio x persona: (1378, 20)
horas tipo servicio x persona: (1378, 20)
tecnologias usadas x persona: (1378, 164)
horas en etapa x persona: (1378, 374)
horas en area x persona: (1378, 44)
pendientes en tipos de servicio x persona: (1378, 20)
riesgos en tipos de servicio x persona: (1378, 20)
Duración de Consulta (seg): 0:00:00.000999


In [ ]:
time_start = datetime.datetime.now()

temp_dataset_1 = pd.merge(pv_horas_tiposervicio_persona, pd.merge(
    df_facturas_persona, df_nofacturas_persona, left_on='PERSONA', right_on='PERSONA'), left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(
    temp_dataset_1, pv_horas_tiposervicio_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(
    temp_dataset_2, pv_tecno_asig_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(
    temp_dataset_1, pv_horas_etapa_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(
    temp_dataset_2, pv_horas_area_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(
    temp_dataset_1, pv_pend_tiposerv_persona, left_on='PERSONA', right_on='PERSONA')
df_dataset_retirados = pd.merge(
    temp_dataset_2, pv_ries_tiposerv_persona, left_on='PERSONA', right_on='PERSONA')
vc_dataset_retirados = df_dataset_retirados.values.tolist()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

print(len(df_dataset_retirados))
print(df_dataset_retirados.head())
print(vc_dataset_retirados[:1])


Duración de Consulta (seg): 0:00:00.399894
1378
   PERSONA  AC_x    ADM_x  AI_x   AP_x   CO_x  CU_x  GC_x  HD_x  IN_x  ...  \
0        2   0.0     0.00   0.0    0.0    0.0     0     0   0.0   0.0  ...   
1        7   0.0    28.00   0.0    0.0    0.0     0     0   0.0   0.0  ...   
2        9   0.0   929.00   0.0  172.5  225.0     0     0   0.0   0.0  ...   
3       10   0.0   217.50   0.0    0.0    0.0     0     0   0.0   0.0  ...   
4       12   0.0  2172.39   0.0    0.0    0.0     0     0   0.0   0.0  ...   

   NE_y  OP_y  PD_y  PM_y  PR_y  RE_y  SE_y  SM_y  SO_y  TD_y  
0     0     0     0     0     0     0     0     0     0     0  
1     0     0     0     0     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     0     0  
3     0     0     0     0     0     0     0     0     0     0  
4     0     0     0     0     0     0     0     0     0     0  

[5 rows x 665 columns]
[[2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

### Consultas Personas Vigentes

In [ ]:
# numero de asignaciones facturadas x persona
time_start = datetime.datetime.now()

cur = connection.cursor()
cur.execute("\
SELECT P.CONSECUTIVO AS PERSONA, CASE WHEN  H.ASIGNACIONES_FACTURADAS IS NULL THEN 0 ELSE H.ASIGNACIONES_FACTURADAS END AS ASIGNACIONES_FACTURADAS \
FROM PERSONAS P \
LEFT JOIN ( \
SELECT P.CONSECUTIVO AS PERSONA, COUNT(A.TARIFA_HORA) AS ASIGNACIONES_FACTURADAS \
FROM ASIGNACIONES A \
INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO = AH.CON_ASIG \
INNER JOIN PERSONAS P                 ON P.CONSECUTIVO = A.CON_PERSONA \
WHERE AH.FECHA_FIN > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
AND   AH.FECHA_FIN < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
AND   AH.FECHA_FIN > AH.FECHA_INICIO \
AND   A.TARIFA_HORA > 0 \
GROUP BY P.CONSECUTIVO) H ON P.CONSECUTIVO = H.PERSONA \
WHERE P.FECHA_RETIRO IS NULL \
ORDER BY P.CONSECUTIVO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_facturas_persona = pd.DataFrame(
    res, columns=['PERSONA', 'ASIGNACIONES_FACTURADAS'])
df_facturas_persona.head(2)


Duración de Consulta (seg): 0:00:00.885815


,PERSONA,ASIGNACIONES_FACTURADAS
0,6,0
1,8,0


In [ ]:
# numero de asignaciones no facturadas x persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT P.CONSECUTIVO AS PERSONA, CASE WHEN  H.ASIGNACIONES_NO_FACTURADAS IS NULL THEN 0 ELSE H.ASIGNACIONES_NO_FACTURADAS END AS ASIGNACIONES_NO_FACTURADAS \
FROM PERSONAS P \
LEFT JOIN ( \
SELECT P.CONSECUTIVO AS PERSONA, COUNT(A.TARIFA_HORA) AS ASIGNACIONES_NO_FACTURADAS \
FROM ASIGNACIONES A \
INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO = AH.CON_ASIG \
INNER JOIN PERSONAS P                 ON P.CONSECUTIVO = A.CON_PERSONA \
WHERE AH.FECHA_FIN > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
AND   AH.FECHA_FIN < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
AND   AH.FECHA_FIN > AH.FECHA_INICIO \
AND   A.TARIFA_HORA = 0 \
GROUP BY P.CONSECUTIVO) H ON P.CONSECUTIVO = H.PERSONA \
WHERE P.FECHA_RETIRO IS NULL \
ORDER BY P.CONSECUTIVO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_nofacturas_persona = pd.DataFrame(
    res, columns=['PERSONA', 'ASIGNACIONES_NO_FACTURADAS'])
df_nofacturas_persona.head(2)


Duración de Consulta (seg): 0:00:00.718139


,PERSONA,ASIGNACIONES_NO_FACTURADAS
0,6,15
1,8,12


In [ ]:
# Asignaciones de personas por tipos de servicio
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.CANTIDAD_TIPOS_SERVICIO IS NULL THEN 0 ELSE B.CANTIDAD_TIPOS_SERVICIO END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
	AND C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.TIPO AS TIPO_SERVICIO, COUNT(S.TIPO) AS CANTIDAD_TIPOS_SERVICIO \
	FROM ASIGNACIONES A \
	INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO  = AH.CON_ASIG \
	INNER JOIN PERSONAS P                 ON A.CON_PERSONA  = P.CONSECUTIVO \
	INNER JOIN SERVICIOS S                ON A.COD_SERVICIO = S.CODIGO_SERVICIO \
	WHERE P.ESTADO = 1 \
	AND AH.FECHA_FIN > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE),-14) \
	AND   AH.FECHA_FIN < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE),-2) \
	AND   AH.FECHA_FIN >= AH.FECHA_INICIO \
	AND   P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO, S.TIPO ) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA,A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_tiposervicio_persona = pd.DataFrame(
    res, columns=['PERSONA', 'TIPO_SERVICIO', 'CANTIDAD_TIPOS_SERVICIO'])
pv_tiposervicio_persona = pd.pivot_table(df_tiposervicio_persona, values='CANTIDAD_TIPOS_SERVICIO', index=[
                                         'PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_tiposervicio_persona.head(2)


Duración de Consulta (seg): 0:00:09.831264


TIPO_SERVICIO,AC,ADM,AI,AP,CO,CU,GC,HD,IN,LI,NE,OP,PD,PM,PR,RE,SE,SM,SO,TD
PERSONA,,,,,,,,,,,,,,,,,,,,
6,0,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8,1,16,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0


In [ ]:
# Horas de personas por tipos de servicio
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.HORAS_TIPO_SERVICIO IS NULL THEN 0 ELSE B.HORAS_TIPO_SERVICIO END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
	AND C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT \
	P.CONSECUTIVO          AS PERSONA, \
	S.TIPO                 AS TIPO_SERVICIO, \
	SUM(EA.DURACION_HORAS) AS HORAS_TIPO_SERVICIO \
	FROM EJECUCION_ACTIVIDADES EA \
	INNER JOIN ENTREGABLES_SERVICIO ES ON EA.CON_ENTREGABLE_SERV = ES.CONSECUTIVO \
	INNER JOIN SERVICIOS S             ON ES.COD_SERVICIO        = S.CODIGO_SERVICIO \
	INNER JOIN PERSONAS  P             ON EA.CONSECUTIVO_PERSONA = P.CONSECUTIVO \
	WHERE P.ESTADO = 1 \
	AND EA.HORA_FINALIZACION > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
	AND   EA.HORA_FINALIZACION < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE) , -2) \
	GROUP BY P.CONSECUTIVO, S.TIPO ) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA,A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_horas_tiposervicio_persona = pd.DataFrame(
    res, columns=['PERSONA', 'TIPO_SERVICIO', 'HORAS_TIPO_SERVICIO'])
pv_horas_tiposervicio_persona = pd.pivot_table(df_horas_tiposervicio_persona, values='HORAS_TIPO_SERVICIO', index=[
                                               'PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_horas_tiposervicio_persona.head(2)


Duración de Consulta (seg): 0:00:14.067172


TIPO_SERVICIO,AC,ADM,AI,AP,CO,CU,GC,HD,IN,LI,NE,OP,PD,PM,PR,RE,SE,SM,SO,TD
PERSONA,,,,,,,,,,,,,,,,,,,,
6,0.0,894.50,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0
8,0.0,1036.57,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,1.0,0,0,0.0,0.0,0.0,0.0


In [ ]:
# Tecnologias usada en asignaciones por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.HERRAMIENTA, CASE WHEN B.VECES_HERRAMIENTA_ASIGNADA IS NULL THEN 0 ELSE B.VECES_HERRAMIENTA_ASIGNADA END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA,H.NOMBRE AS HERRAMIENTA \
	FROM HERRAMIENTAS H	 \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
	AND   P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO,H.NOMBRE) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO  AS PERSONA,H.NOMBRE  AS HERRAMIENTA,COUNT(AH.CONSECUTIVO) AS VECES_HERRAMIENTA_ASIGNADA \
	FROM HERRAMIENTAS_UTILIZADAS HU \
	INNER JOIN HERRAMIENTAS H             ON HU.CON_HERRAMIENTAS = H.CONSECUTIVO \
	INNER JOIN SERVICIOS S                ON HU.COD_SERVICIO     = S.CODIGO_SERVICIO \
	INNER JOIN ASIGNACIONES A             ON S.CODIGO_SERVICIO   = A.COD_SERVICIO \
	INNER JOIN ASIGNACIONES_HISTORICOS AH ON A.CONSECUTIVO       = AH.CON_ASIG \
	INNER JOIN PERSONAS P                 ON P.CONSECUTIVO       = A.CON_PERSONA \
	WHERE P.ESTADO = 1 \
	AND   AH.FECHA_FIN > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
	AND   AH.FECHA_FIN < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
	AND   AH.FECHA_FIN > AH.FECHA_INICIO \
	AND   P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO, H.NOMBRE) B ON A.PERSONA = B.PERSONA AND  A.HERRAMIENTA = B.HERRAMIENTA \
ORDER BY A.PERSONA, A.HERRAMIENTA")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_tecno_asig_persona = pd.DataFrame(
    res, columns=['PERSONA', 'HERRAMIENTA', 'VECES_HERRAMIENTA_ASIGNADA'])
pv_tecno_asig_persona = pd.pivot_table(df_tecno_asig_persona, values='VECES_HERRAMIENTA_ASIGNADA', index=[
                                       'PERSONA'], columns='HERRAMIENTA', fill_value=0)
pv_tecno_asig_persona.head(2)


Duración de Consulta (seg): 0:01:19.896621


HERRAMIENTA,.NET,.NET Core,ANGULAR,AWS,Activemq,Alibaba,Android / Java,Android / Kotlin,Angular,Apache Airflow,...,Travis CI,TypeScript,Vue,WEBLOGIC,Websphere,Websphere Message Broker,Wordpress,Xamarin,iOS / Objetive C,iOS / Swift
PERSONA,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Horas por etapa por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.ETAPA, CASE WHEN B.HORAS_ETAPA IS NULL THEN 0 ELSE B.HORAS_ETAPA END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, E.CODIGO_ETAPA  AS ETAPA \
	FROM ETAPAS  E \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
    AND    P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO, E.CODIGO_ETAPA) A \
LEFT JOIN ( \
	SELECT \
	P.CONSECUTIVO          AS PERSONA, \
	EA.CODIGO_ETAPA        AS ETAPA, \
	SUM(EA.DURACION_HORAS) AS HORAS_ETAPA \
	FROM EJECUCION_ACTIVIDADES EA \
	INNER JOIN PERSONAS P ON EA.CONSECUTIVO_PERSONA = P.CONSECUTIVO \
	WHERE P.ESTADO = 1 \
	AND   EA.HORA_FINALIZACION > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
	AND   EA.HORA_FINALIZACION < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
	GROUP BY P.CONSECUTIVO, EA.CODIGO_ETAPA ) B ON A.PERSONA = B.PERSONA AND A.ETAPA = B.ETAPA \
ORDER BY A.PERSONA, A.ETAPA")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_horas_etapa_persona = pd.DataFrame(
    res, columns=['PERSONA', 'ETAPA', 'HORAS_ETAPA'])
df_horas_etapa_persona['ETAPA'] = df_horas_etapa_persona['ETAPA'] .fillna(
    'SIN_ETAPA')
pv_horas_etapa_persona = pd.pivot_table(df_horas_etapa_persona, values='HORAS_ETAPA', index=[
                                        'PERSONA'], columns='ETAPA', fill_value=0)
pv_horas_etapa_persona.head(2)


Duración de Consulta (seg): 0:02:58.397140


ETAPA,ACTCOM,ADMB,ADMBD,ADMCAP,ADMON,ADMSEG,ADMSER,ADMSO,ALEMAN,ANAID,...,TRIBUT,TRINS,TRINSM,TRINT,TRMIG,TRREV,VAC,VAR,VENPRO,VISCLI
PERSONA,,,,,,,,,,,,,,,,,,,,,
6,0,0,0.0,0,0.0,0,0.0,0,0,0,...,0.0,0.0,0,0,0.0,0.0,56.0,0,0.0,0.0
8,0,0,0.0,0,0.0,0,0.0,0,0,0,...,0.0,0.0,0,0,0.0,0.0,120.0,0,0.0,0.0


In [ ]:
# Horas en area por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT B.PERSONA, B.AREA, CASE WHEN C.HORAS_AREA IS NULL THEN 0 ELSE C.HORAS_AREA END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, A.CONSECUTIVO  AS AREA \
	FROM AREAS  A \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
    AND    P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO,A.CONSECUTIVO  ) B \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.CON_AREA  AS AREA, SUM(EA.DURACION_HORAS) AS HORAS_AREA \
	FROM EJECUCION_ACTIVIDADES EA \
	INNER JOIN ENTREGABLES_SERVICIO ES ON EA.CON_ENTREGABLE_SERV = ES.CONSECUTIVO \
	INNER JOIN PERSONAS P              ON EA.CONSECUTIVO_PERSONA = P.CONSECUTIVO \
	INNER JOIN SERVICIOS S             ON ES.COD_SERVICIO  = S.CODIGO_SERVICIO \
	WHERE P.ESTADO = 1 \
	AND   EA.HORA_FINALIZACION > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
	AND   EA.HORA_FINALIZACION < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
	GROUP BY P.CONSECUTIVO, S.CON_AREA ) C ON B.PERSONA = C.PERSONA AND C.PERSONA =  B.AREA \
ORDER BY B.PERSONA, B.AREA")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_horas_area_persona = pd.DataFrame(
    res, columns=['PERSONA', 'AREA', 'HORAS_AREA'])
pv_horas_area_persona = pd.pivot_table(df_horas_area_persona, values='HORAS_AREA', index=[
                                       'PERSONA'], columns='AREA', fill_value=0)
pv_horas_area_persona.head(2)


Duración de Consulta (seg): 0:00:24.456941


AREA,1,2,3,4,5,6,7,8,9,24,...,53,54,55,56,58,59,60,61,62,63
PERSONA,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,127.714286,0,0.000,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0.000000,0,172.845,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Cantidad de pendientes en tipo de servicio por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.CANTIDAD_PENDIENTES IS NULL THEN 0 ELSE B.CANTIDAD_PENDIENTES END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
	AND    C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.TIPO AS TIPO_SERVICIO, COUNT(PE.CODIGO_PENDIENTE) AS CANTIDAD_PENDIENTES \
	FROM ASIGNACIONES_HISTORICOS AH \
	INNER JOIN ASIGNACIONES A  ON A.CONSECUTIVO = AH.CON_ASIG \
	INNER JOIN PERSONAS     P  ON A.CON_PERSONA = P.CONSECUTIVO \
	INNER JOIN PENDIENTES   PE ON A.COD_SERVICIO = PE.CODIGO_SERVICIO \
	INNER JOIN SERVICIOS    S  ON PE.CODIGO_SERVICIO = S.CODIGO_SERVICIO \
	WHERE P.ESTADO = 1 \
	AND   AH.FECHA_FIN > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
	AND   AH.FECHA_FIN < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
	AND   AH.FECHA_FIN >= AH.FECHA_INICIO \
	AND   PE.FECHA_DETECCION > ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) \
	AND   PE.FECHA_DETECCION < ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
	GROUP BY P.CONSECUTIVO, S.TIPO) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA,A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_pend_tiposerv_persona = pd.DataFrame(
    res, columns=['PERSONA', 'TIPO_SERVICIO', 'CANTIDAD_PENDIENTES'])
pv_pend_tiposerv_persona = pd.pivot_table(df_pend_tiposerv_persona, values='CANTIDAD_PENDIENTES', index=[
                                          'PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_pend_tiposerv_persona.head(2)


Duración de Consulta (seg): 0:00:14.550336


TIPO_SERVICIO,AC,ADM,AI,AP,CO,CU,GC,HD,IN,LI,NE,OP,PD,PM,PR,RE,SE,SM,SO,TD
PERSONA,,,,,,,,,,,,,,,,,,,,
6,0,2049,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8,0,543,0,0,0,0,0,0,0,0,0,0,0,19,0,0,0,0,0,0


In [ ]:
# Cantidad de riesgos en tipo de servicio por persona
time_start = datetime.datetime.now()

cur.execute("\
SELECT A.PERSONA, A.TIPO_SERVICIO, CASE WHEN B.CANTIDAD_RIESGOS IS NULL THEN 0 ELSE B.CANTIDAD_RIESGOS END \
FROM ( \
	SELECT P.CONSECUTIVO AS PERSONA, C.RV_LOW_VALUE AS TIPO_SERVICIO \
	FROM CG_REF_CODES C \
	CROSS JOIN PERSONAS P \
	WHERE P.ESTADO = 1 \
	AND    C.RV_DOMAIN = 'TIPO_PROYECTO' \
	AND    P.FECHA_RETIRO IS NULL \
	GROUP BY P.CONSECUTIVO,C.RV_LOW_VALUE ) A \
LEFT JOIN ( \
	SELECT P.CONSECUTIVO AS PERSONA, S.TIPO AS TIPO_SERVICIO, COUNT(RS.CONSECUTIVO) AS CANTIDAD_RIESGOS \
	FROM ASIGNACIONES_HISTORICOS AH \
	INNER JOIN ASIGNACIONES A      ON A.CONSECUTIVO = AH.CON_ASIG \
	INNER JOIN PERSONAS     P      ON A.CON_PERSONA = P.CONSECUTIVO \
	INNER JOIN RIESGOS_SERVICIO RS ON A.COD_SERVICIO = RS.CODIGO_SERVICIO \
	INNER JOIN SERVICIOS    S      ON RS.CODIGO_SERVICIO = S.CODIGO_SERVICIO \
	WHERE P.ESTADO = 1 \
	AND   AH.FECHA_FIN BETWEEN ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14) AND ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
	AND   AH.FECHA_FIN > AH.FECHA_INICIO \
	AND   RS.FECHA_REGISTRO BETWEEN ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -14)  AND ADD_MONTHS(CAST(CURRENT_TIMESTAMP AS DATE), -2) \
	GROUP BY P.CONSECUTIVO, S.TIPO ) B ON A.PERSONA = B.PERSONA AND A.TIPO_SERVICIO = B.TIPO_SERVICIO \
ORDER BY A.PERSONA, A.TIPO_SERVICIO")
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_ries_tiposerv_persona = pd.DataFrame(
    res, columns=['PERSONA', 'TIPO_SERVICIO', 'CANTIDAD_RIESGOS'])
pv_ries_tiposerv_persona = pd.pivot_table(df_ries_tiposerv_persona, values='CANTIDAD_RIESGOS', index=[
                                          'PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
pv_ries_tiposerv_persona.head(2)


Duración de Consulta (seg): 0:00:10.087617


TIPO_SERVICIO,AC,ADM,AI,AP,CO,CU,GC,HD,IN,LI,NE,OP,PD,PM,PR,RE,SE,SM,SO,TD
PERSONA,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
time_start = datetime.datetime.now()

print('asignaciones facturadas x persona:', df_facturas_persona.shape)
print('asignaciones no facturadas x persona:', df_nofacturas_persona.shape)
print('numero tipos servicio x persona:', pv_tiposervicio_persona.shape)
print('horas tipo servicio x persona:', pv_horas_tiposervicio_persona.shape)
print('tecnologias usadas x persona:', pv_tecno_asig_persona.shape)
print('horas en etapa x persona:', pv_horas_etapa_persona.shape)
print('horas en area x persona:', pv_horas_area_persona.shape)
print('pendientes en tipos de servicio x persona:',
      pv_pend_tiposerv_persona.shape)
print('riesgos en tipos de servicio x persona:', pv_ries_tiposerv_persona.shape)

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")


asignaciones facturadas x persona: (593, 2)
asignaciones no facturadas x persona: (593, 2)
numero tipos servicio x persona: (590, 20)
horas tipo servicio x persona: (590, 20)
tecnologias usadas x persona: (590, 164)
horas en etapa x persona: (590, 374)
horas en area x persona: (590, 44)
pendientes en tipos de servicio x persona: (590, 20)
riesgos en tipos de servicio x persona: (590, 20)
Duración de Consulta (seg): 0:00:00.001998


In [ ]:
time_start = datetime.datetime.now()

temp_dataset_1 = pd.merge(pv_horas_tiposervicio_persona, pd.merge(
    df_facturas_persona, df_nofacturas_persona, left_on='PERSONA', right_on='PERSONA'), left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(
    temp_dataset_1, pv_horas_tiposervicio_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(
    temp_dataset_2, pv_tecno_asig_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(
    temp_dataset_1, pv_horas_etapa_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_1 = pd.merge(
    temp_dataset_2, pv_horas_area_persona, left_on='PERSONA', right_on='PERSONA')
temp_dataset_2 = pd.merge(
    temp_dataset_1, pv_pend_tiposerv_persona, left_on='PERSONA', right_on='PERSONA')
df_dataset_vigentes = pd.merge(
    temp_dataset_2, pv_ries_tiposerv_persona, left_on='PERSONA', right_on='PERSONA')
vc_dataset_vigentes = df_dataset_vigentes.values.tolist()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

print(len(df_dataset_vigentes))
print(vc_dataset_vigentes[:1])


In [ ]:
if connection:
    connection.close()


In [ ]:
df_dataset_vigentes
vc_dataset_vigentes
df_dataset_retirados
vc_dataset_retirados


In [ ]:
df_dataset_retirados.head(5)
df_dataset_retirados.to_csv('personas_rindex=False)


In [14]:
###tHOMAS MODIFY THIS TO BE PERSONAS




# Cantidad de pendientes en tipo de servicio por persona
time_start = datetime.datetime.now()

cur.execute("SELECT CON_EMPRESA, CONSECUTIVO FROM PERSONAS WHERE FECHA_RETIRO IS NOT NULL") #ALL PEOPLE WHO HAVE LEFT
res = cur.fetchall()

query_time = datetime.datetime.now() - time_start
print(f"Duración de Consulta (seg): {query_time}")

df_persona_table = pd.DataFrame(
    res, columns=['CON_EMPRESA','CONSECUTIVO'])
#pv_pend_tiposerv_persona = pd.pivot_table(df_pend_tiposerv_persona, values='CANTIDAD_PENDIENTES', index=[
 #                                         'PERSONA'], columns='TIPO_SERVICIO', fill_value=0)
#pv_pend_tiposerv_persona.head(2)

print(df_persona_table)

Duración de Consulta (seg): 0:00:03.427307
      CON_EMPRESA  CONSECUTIVO
0            84.0          553
1            84.0          560
2            84.0          566
3            84.0          567
4            84.0          568
...           ...          ...
1373         84.0         2304
1374         84.0         2314
1375         84.0         2315
1376         84.0         2486
1377         84.0         2513

[1378 rows x 2 columns]
